In [6]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 内存中的短时记忆

<hr>

In [1]:
from langchain.memory import ConversationBufferMemory

# 存储数据到内存
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是大头同学！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")
# 加载内存中数据
memory.load_memory_variables({})

{'history': 'Human: 你好，我是大头同学！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [2]:
# 加载内存中数据
memory.buffer

'Human: 你好，我是大头同学！\nAI: 你好，我是AI，有什么可以帮助你的吗？'

In [17]:
#实现一个最近的对话窗口，超过窗口条数的对话(一对input+output算一组对话)将被删除
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "你好，我是人类!"}, {"output": "你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input": "我想吃鸡肉"}, {"output": "好的，我帮你找找鸡肉的做法"})

# k=1,内存只会保存最新的一条数据
memory.load_memory_variables({})

{'history': 'Human: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

# 构建记忆实体概念清单
在存储的同时, 会把对话中的实体单独存储起来
<hr>

In [28]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory

llm = OpenAI(temperature=0)

# 从对话的内容中,提取实体概念
memory = ConversationEntityMemory(llm=llm)
_input = {
    "input": "胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}
ans = memory.load_memory_variables(_input)
print(ans)
memory.save_context(
    _input,
    {
        "output": "听起来很刺激，我也想加入他们！"
    }
)

{'history': '', 'entities': {'胡八一': '', '王胖子': '', '雪莉杨': '', '盗墓铁三角': ''}}


In [29]:
memory.load_memory_variables({"input": "铁三角是谁?"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {}}

# 使用知识图谱构建记忆

In [30]:
from langchain.llms import OpenAI
from langchain.memory import ConversationKGMemory

llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm, return_messages=True)

memory.save_context(
    {"input": "帮我找一下tomie"},
    {"output": "对不起请问什么是tomie？"}
)

memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)

In [31]:
memory.load_memory_variables({"input": "tomie是谁?"})

{'history': [SystemMessage(content='On tomie: tomie is a training instructor.')]}

In [32]:
# 提取实体
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [33]:
# 获取三元知识组: 主题,动作,目标
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='tomie', predicate='最喜欢', object_='打游戏')]

# 长对话在内存中的处理方式：总结摘要以及token计算

## 总结摘要
会把之前的会话内容做总结摘要

In [34]:
from langchain.memory import ConversationSummaryMemory
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input": "帮我找一下tomie"},
    {"output": "对不起请问什么是tomie？"}
)
memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)

In [35]:
memory.load_memory_variables({})

{'history': '\nThe human asks the AI to help find something called "tomie". The AI apologizes and asks what "tomie" is. The human explains that tomie is a training instructor. The AI acknowledges and says it understands.'}

In [36]:
messages = memory.chat_memory.messages
#print(messages)
memory.predict_new_summary(messages, "")

'\nThe human asks the AI to help find information about "tomie". The AI asks for clarification on what "tomie" is. The human explains that "tomie" is a training instructor. The AI acknowledges and understands.'

In [37]:
#使用ChatMessageHistory来快速获得对话摘要
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain.llms import OpenAI

history = ChatMessageHistory()
history.add_user_message("你好，我是人类！")
history.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

memory = ConversationSummaryMemory.from_messages(
    llm=OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True
)

In [38]:
#memory.buffer
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human introduces themselves as a human and the AI introduces itself as AI Xiaowanzi, asking if there is anything it can help with.')]}

## ConversationSummaryBufferMemory
- 当对话持续进行且对话内容很多的时候可以使用ConversationSummaryBufferMemory来存储对话摘要
- 这是一种非常有用的方式,它会根据token的数量来自动判断是否需要进行摘要
- 当token数量超过阈值的时候,会自动进行摘要
- 在缓冲区中,会保留最近的k条对话, 比较久的对话会总结为摘要记录,然后删除会话

In [44]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True
)
memory.save_context(
    {"input": "帮我找一下tomie"},
    {"output": "对不起请问什么是tomie？"}
)
memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)
memory.save_context(
    {"input": "今天他要讲一门关于RAG的课程"},
    {"output": "好的，我知道了。需要RAG的资料吗？"}
)


In [45]:
# 可以看到之前的内容变成了摘要,后续新的会话还存在
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe human asks the AI to help find Tomie. The AI asks for clarification on who or what Tomie is. The human explains that Tomie is a training instructor. The AI acknowledges and understands.'),
  HumanMessage(content='今天他要讲一门关于RAG的课程'),
  AIMessage(content='好的，我知道了。需要RAG的资料吗？')]}

## Conversation Token Buffer使用token长度来决定什么时候刷新内存

In [48]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context(
    {"input": "帮我找一下tomie"},
    {"output": "对不起请问什么是tomie？"}
)
memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)
memory.save_context(
    {"input": "今天他要讲一门关于RAG的课程"},
    {"output": "好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input": "不需要资料了，谢谢"},
    {"output": "好的，那我就不打扰你了。"}
)


In [49]:
# 可以看到只保留了最新100个token的对话累哦人
memory.load_memory_variables({})

{'history': 'Human: tomie是一个培训讲师\nAI: 好的，我知道了。\nHuman: 今天他要讲一门关于RAG的课程\nAI: 好的，我知道了。需要RAG的资料吗？\nHuman: 不需要资料了，谢谢\nAI: 好的，那我就不打扰你了。'}

# 长时记忆的是实现方式

- 通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话
- 也可以使用其他的存储介质(redis,mongo,es等),langchain都有内置的实现类

In [50]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

memory = ConversationBufferMemory()
memory.save_context(
    {"input": "帮我找一下tomie"},
    {"output": "对不起请问什么是tomie？"}
)
memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)
memory.save_context(
    {"input": "今天他要讲一门关于RAG的课程"},
    {"output": "好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input": "不需要资料了，谢谢"},
    {"output": "好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
# 存储在本地,会有一个
FAISS.save_local(vectorstore, "faiss_db_file")

In [54]:
FAISS.load_local("faiss_db_file", OpenAIEmbeddings(model="text-embedding-3-small"),
                 allow_dangerous_deserialization=True).similarity_search("tomie是什么职业?")

[Document(page_content='Human: 今天他要讲一门关于RAG的课程'),
 Document(page_content='AI: 好的，我知道了。需要RAG的资料吗？'),
 Document(page_content='Human: tomie是一个培训讲师'),
 Document(page_content='AI: 好的，那我就不打扰你了。')]

In [55]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

r1 = FAISS.load_local("faiss_db_file", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
memory2 = VectorStoreRetrieverMemory(
    retriever=r1.as_retriever(search_kwargs={"k": 1})
)
memory2.load_memory_variables({"prompt": "tomie是什么职业?"})

{'history': 'Human: tomie是一个培训讲师'}